In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-244483
Azure region: eastus2
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-244483


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_cluster_name = "demo-aml-compute"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, use this cluster that was found.")
except:
    print("Creating new cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Creating new cluster...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-11-27T17:45:42.471000+00:00', 'errors': None, 'creationTime': '2023-11-27T17:45:34.193257+00:00', 'modifiedTime': '2023-11-27T17:45:43.884321+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 0.2, 0.5, 0.7, 1.0),
    '--max_iter': choice(range(10,110,10))
    }
)  ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, # specifies the allowable slack as a ratio
                      evaluation_interval=2, # frequency for applying the policy
                      delay_evaluation=5) # delays the first policy evaluation for a specified number of intervals### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
est =SKLearn(source_directory = './', entry_script='train.py', compute_target=compute_target, vm_size='STANDARD_D2_V2') ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4
                                    )### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

#hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7c9118b7-8eff-49fa-b594-91e08f9447f7
Web View: https://ml.azure.com/runs/HD_7c9118b7-8eff-49fa-b594-91e08f9447f7?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-244483/workspaces/quick-starts-ws-244483&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-11-27T17:46:02.357261][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-11-27T17:46:02.7482550Z][SCHEDULER][INFO]Scheduling job, id='HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_0' 
[2023-11-27T17:46:02.8660874Z][SCHEDULER][INFO]Scheduling job, id='HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_1' 
[2023-11-27T17:46:02.9894835Z][SCHEDULER][INFO]Scheduling job, id='HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_2' 
[2023-11-27T17:46:03.098244][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-11-27T17:46:03.1566923Z][SCHEDULER][INFO]Scheduling job, id='HD_7c9118b7-8eff-49fa-b594-91e08f9447

{'runId': 'HD_7c9118b7-8eff-49fa-b594-91e08f9447f7',
 'target': 'demo-aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-11-27T17:46:01.736925Z',
 'endTimeUtc': '2023-11-27T17:54:35.142699Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4365784a-4708-45e0-8a36-ab5fdb0158d4',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '60',
  'best_child_run_id': 'HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_1',
  'score': '0.9108750632271118',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCl

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("best run details :", hyperdrive_best_run.get_details())
print('\n')
print("best run file names :", hyperdrive_best_run.get_file_names())
print('\n')
print("best run metrics :", hyperdrive_best_run.get_metrics())

model = hyperdrive_best_run.register_model(model_name='hyperdrive_log_reg_best_run', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy" : hyperdrive_best_run.get_metrics()["Accuracy"]})

best run details : {'runId': 'HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_1', 'target': 'demo-aml-compute', 'status': 'Completed', 'startTimeUtc': '2023-11-27T17:51:47.144692Z', 'endTimeUtc': '2023-11-27T17:53:41.195982Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '4365784a-4708-45e0-8a36-ab5fdb0158d4', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.7', '--max_iter', '70'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'demo-aml-compute', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'def

In [7]:
print('Best Run Id: ', hyperdrive_best_run.id)
print('Accuracy:',hyperdrive_best_run.get_metrics()['Accuracy'])
hyperdrive_best_run

Best Run Id:  HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_1
Accuracy: 0.9108750632271118


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_7c9118b7-8eff-49fa-b594-91e08f9447f7_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###


        

In [10]:
from sklearn.model_selection import train_test_split

# split data into train and test sets - 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=123, stratify = y)
import pandas as pd

# Create a new dataframe with only training data

df_train = pd.concat([x_train,y_train], axis=1)
df_train.reset_index(drop=True, inplace=True)


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    max_concurrent_iterations=4,
    max_cores_per_iteration=-1,
    featurization='auto',
    iterations=30,
    enable_early_stopping=True,
    debug_log = 'automl_errors.log',
    training_data=x,
    label_column_name='y',
    n_cross_validations=2)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###

experiment_name = 'automl-experiment'

experiment = Experiment(ws, experiment_name)
automl_run = experiment.submit(config=automl_config, show_output=True)
automl_run

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_756b470d-9fee-40e7-a1df-e15f63bcb5b4,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/11/27 18:06:36 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_756b470d-9fee-40e7-a1df-e15f63bcb5b4,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
# from azureml.widgets import RunDetails
RunDetails(automl_run).show()

automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_756b470d-9fee-40e7-a1df-e15f63bcb5b4,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2584                          |1                               |23065                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_756b470d-9fee-40e7-a1df-e15f63bcb5b4',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2023-11-27T18:00:33.043314Z',
 'endTimeUtc': '2023-11-27T18:32:12.078834Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-244483","workspace_name":"quick-starts-ws-244483","region":"eastus2","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enab

In [14]:
# Retrieve model

best_run, fitted_model = automl_run.get_output()

# get name of files of best_run
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/192cb6fb/classes.interpret.json',
 'explanation/192cb6fb/expected_values.interpret.json',
 'explanation/192cb6fb/features.interpret.json',
 'explanation/192cb6fb/global_names/0.interpret.json',
 'explanation/192cb6fb/global_rank/0.interpret.json',
 'explanation/192cb6fb/global_values/0.interpret.json',
 'explanation/192cb6fb/local_importance_values_sparse.interpret.json',
 'explanation/192cb6fb/per_class_names/0.interpret.json',
 'explanation/192cb6fb/per_class_rank/0.interpret.json',
 'explanation/192cb6fb/per_class_values/0.interpret.json',
 'explanation/192cb6fb/rich_metadata.interpret.json',
 'explanation/192cb6fb/true_ys_viz.interpret.json',
 'explanation/192cb6fb/visualization_dict.interpret.json',
 'explanation/192cb6fb/ys_pred_proba_viz.interpret.json',
 'explanation/192cb6fb/ys_pred_viz.interpret.json',
 'explanation/646420af/classes.interpret.json',
 'explanation/646420af/eval_data_viz.interpret.json',
 'explanation/646420

In [15]:
# save best model
best_run.register_model(model_name = "model.pkl", model_path = './outputs/')
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('21', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('

In [16]:
fitted_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassif

# Cleaning Up Cluster

In [18]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

